In [2]:
import torch
from torchvision.models import resnet18, ResNet18_Weights

# resent18 모델 불러오기
model = resnet18(weights=ResNet18_Weights.DEFAULT)
# data = 3채널, 높이 64, 넓이 64
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000) # 라벨 데이터는 (1,1000) 모양

# 순전파 단계(forward)
prediction = model(data) # prediction

loss = (prediction-labels).sum()  # loss 계산
loss.backward() # 역전파 단계(backward)

# 옵티마이저 불러오기
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9) # 0.1 학습률, 0.9 모멘텀
# 모멘텀은 현재 지점의 경사도 + 이전까지의 진행 방향(관성)을 함께 고려함/
# 0.9는 이전 업데이트 방향을 90% 반영하겠다. 학습 속도 향상 + 작은 언덕에서 빠져나올 확률이 큼 + 안정적인 학습
optim.step() # 경사하강법(gradient descent)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\joohy/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth
100.0%


In [3]:
import torch

a = torch.tensor([2.,3.], requires_grad=True) # requires_grad 모든 연산 추적 여부
b = torch.tensor([6.,4.], requires_grad=True)

# 새로운 텐서 만들기
Q = 3*a**3-b**2

# Q가 오차(error)라고 가정, Q에 대해서 .backward()를 호출하면, autograd는 변화도를 계산하고 .grad 속성(atiiribute)에 저장한다.
external_grad = torch.tensor([1.,1.])
Q.backward(gradient=external_grad) # loss가 스칼라 값이 아니면 gradient 명시해야함

# 변화도는 a.grad 와 b.grad에 저장됨
print(9*a**2 == a.grad)
print(12*b == b.grad)

tensor([True, True])
tensor([False, False])


In [ ]:
# autograd는 연산들의 기록을 방향성 비순환 그래프(DAG에 저장)
# DAG의 leave는 입력 텐서, root는 결과 텐서, 뿌리에서부터 잎까지 추적하면 연쇄 법칙(chain rule)에 따라 변화도를 자동으로 계산

# 순전파 단계에서 autograd는 결과 텐서 계산, DAG에 변화도 기능(gradient function)을 유지한다.
# 역전파 단계에서 DAG 뿌리에서 .backward()가 호출될 때 시작됨, autograd는 각 .grad_fn으로 부터 변화도를 계산하고, 각 텐서의 .grad 속성에 계산 결과를 쌓고(accumulate), 연쇄 법칙을 사용하여, 모든 leaf 텐서들까지 전파한다.

# requires_grad가 True면 모든 텐서에 대한 연산들을 추적한다. 입력 텐서 중 하나라도 True면 연산 결과 텐서도 변화도를 갖음.
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients?: {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")


# 변화도를 계산하지 않는 매개변수를 일반적으로 고정된 매개변수(frozen parameter)라고 함.
# 변화도가 필요없으면 고정하는 것이 가능. fine-tuning에서 새로운 정답을 예측할 수 있도록 모델의 대부분을 고정한 뒤 일반적으로 분류 계층(classifier layer)만 변경함 (여러 방법 중 하나)
from torch import nn,optim

# 모델 불러오기
model = resnet18(weights=ResNet18_Weights.DEFAULT)

# 신경망의 모든 매개변수를 고정함.
for param in model.parameters():
    param.requires_grad = False

# 10개의 정답을 갖는 새로운 데이터셋으로 모델을 미세조정
# resnet에서 classifier은 마지막 선형 계층(linear layer)인 model.fc으로 초기화
model.fc = nn.Linear(512, 10)

# 마지막 분류기만 최적화 대상으로 삼음
optimizer = optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

Does `a` require gradients?: False
Does `b` require gradients?: True
